figE8 

spatial relationship between cell cluster and ptcs

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl

import scanpy as sc
import squidpy as sq

time = sys.argv[1]

# my color
my_color = ["#93d0fc", "#2aaae1", "#3f5246", "#ad3074", "#E7B10A", 
"#91a3b0", "#d58282", "#418557", "#6dc831", "#eacd01",
"#ff9f22", "#af5b2b", "#2e5073", "#E6D2AA", "#42032C",
"#752f9a", "#721c28", "black"]
my_palette = mpl.colors.ListedColormap(my_color)

fpath = "final_used_data.h5ad"
adata = sc.read(fpath)

main_cluster = list(range(17))
adata = adata[adata.obs["seurat_clusters"].isin(main_cluster) & (adata.obs["hours"]==time)]

pi_cells = pd.read_csv("/public/home/daijichen/01_workdir/02_liyuying_2/00_fig4/fig_pi/01.pi_potato.ps_time_related/ps_time.pi_counts.tab",
                      header=0, index_col=None, sep="\t")
pi_cells = set(pi_cells['cellid'])
adata.obs["seurat_clusters_pi"] = adata.obs.apply(lambda x: str(x['seurat_clusters']) if x['cellid'] not in pi_cells else 'pi', axis = 1)
adata.obs['seurat_clusters_pi'] = adata.obs['seurat_clusters_pi'].astype('category') # to str
adata.obs['seurat_clusters'] = adata.obs['seurat_clusters'].astype('category') # to str

adata.obsm["spatial"] = np.array(adata.obs[['x', 'y']])
# delaunay graph
sq.gr.spatial_neighbors(adata, coord_type="generic", delaunay = True)
sq.gr.nhood_enrichment(adata, cluster_key="seurat_clusters")
sq.pl.nhood_enrichment(adata, cluster_key="seurat_clusters", method="ward", cmap = "coolwarm", save="./" + time + "_nhood_enrich.svg")

In [ ]:
library(ggplot2)
library(Seurat)
library(pheatmap)
library(viridis)
set.seed(12345)
library(circlize)
library(tidyverse)
library(ComplexHeatmap)
#library(monocle)
set.seed(12345)

pi_info <- read.table("inf_pi.tab", header = T, sep="\t")
pi_neighbor <- read.table("pi.neighbor.giotto.tab", header=T, sep="\t")
pi_cell <- pi_info[pi_info$nfeature_10.12=="Y",][["cellid"]]
pi_neighbor_cell <- pi_neighbor[pi_neighbor$nb_cells!="others",][["cell_ID"]]
pi_neighbor_cell <- setdiff(pi_neighbor_cell, pi_cell)

potato.merged <- readRDS("final_used_data.RDS")
potato.merged$pi_nb <- ifelse(potato.merged$cellid %in% pi_cell, "pi", "N")
potato.merged$pi_nb <- ifelse(potato.merged$cellid %in% pi_neighbor_cell, "pi_nb", potato.merged$pi_nb)
potato.merged$pi_nb <- ifelse(potato.merged$inf_stat=="Noninf", "Noninf", potato.merged$pi_nb)
potato.merged$pi_nb <- ifelse(potato.merged$inf_stat=="CK", "CK", potato.merged$pi_nb)

clt_exp <- AverageExpression(potato.merged, group.by="seurat_clusters", return.seurat=TRUE)
pi_exp <- AverageExpression(potato.merged, group.by="pi_nb", return.seurat=TRUE)

exp_mat_clt <- clt_exp@assays$Spatial@data
exp_mat_pi <- pi_exp@assays$Spatial@data

hvg <- VariableFeatures(potato.merged, nfeature=2000)
exp_mat_clt <- exp_mat_clt[hvg, ]
exp_mat_pi <- exp_mat_pi[hvg, ]
merge_mat <- cbind(exp_mat_clt, exp_mat_pi)[, c(as.character(0:16), "pi")]
corr_mat <- cor(merge_mat)

hm <- Heatmap(corr_mat,
        col = c("#440154FF", "#440154FF", "#440154FF", "#440154FF", "#440154FF", "#440154FF", viridis(25)),
        na_col = "grey",
        cluster_rows = T,
        cluster_columns = T,
        #row_split = sub_exp_df$stat,
        column_title = NULL,
        #row_labels = rownames(sub_exp_df),
        show_column_names = T,
        show_row_names=T,
        width = unit(0.5, "snpc"), height = unit(0.5, "snpc"),

        )

pdf(paste0("06.exp_sim_pi_clt.hmap.pdf"),width=10,height=10)
draw(hm)
dev.off()